In [1]:
# pulls from the rotowire

In [2]:
# import the libraries
import requests
import pandas as pd
import json
import glob
from IPython.display import display
from datetime import datetime
import nfl_data_py as nfl
import os

In [3]:
# this imports all modified columns correction and in order 

# define the template URL for API fetching
rotowire_template_url = "https://www.rotowire.com/betting/nfl/tables/nfl-games.php?week={week}"

# define the columns
columns_to_keep = ['gameID', 'year', 'week', 'gameDate', 'gameDateTime', 'homeAway', 'nickname', 'abbr',
                   'oppNickname', 'draftkings_moneyline', 'draftkings_spread', 'draftkings_ou', 'draftkings_score', 
                   'draftkings_oppScore', 'fanduel_moneyline', 'fanduel_spread', 'fanduel_ou', 'fanduel_score', 'fanduel_oppScore',
                   'betrivers_moneyline', 'betrivers_spread', 'betrivers_ou', 'betrivers_score', 'betrivers_oppScore',
                   'mgm_moneyline', 'mgm_spread', 'mgm_ou', 'mgm_score', 'mgm_oppScore'
                  ]

# fetch JSON data from API and filter columns
def import_rotowire_data(week):
    # Format the URL with the given week
    url = rotowire_template_url.format(week=week)
    
    # Fetch the JSON data from the API
    response = requests.get(url)
    
    if response.status_code == 200:
        # Convert the JSON data to a Python object
        json_data = response.json()
        
        if not json_data:
            return None
        
        # Convert the JSON data to a DataFrame
        df = pd.DataFrame(json_data)
        
        # Add the 'week' column manually
        df['week'] = week
        
        # Add the 'year' column derived from the 'gameDate' column
        df['gameDate'] = pd.to_datetime(df['gameDate'], errors='coerce')  # Convert to datetime
        df['year'] = df['gameDate'].dt.year
        
        # Drop all columns that are not in 'columns_to_keep'
        filtered_df = df[columns_to_keep]

        return filtered_df
    else:
        return None

# load missing weeks from local JSON files
def load_json_files():
    # List to hold data for missing weeks
    missing_weeks_data = []

    # Find all JSON files that follow the pattern for week files
    json_files = glob.glob("rotowire_odds_lines_week_*.json")
    
    for json_file in json_files:
        # Extract the week number from the file name
        week_number = int(json_file.split('_')[-1].replace('.json', ''))
        
        # Read the JSON file
        with open(json_file, 'r') as f:
            json_data = json.load(f)
        
        # Convert the JSON data to a DataFrame
        df = pd.DataFrame(json_data)
        
        # Add the week number and derive the year from the gameDate
        df['week'] = week_number
        df['gameDate'] = pd.to_datetime(df['gameDate'], errors='coerce')
        df['year'] = df['gameDate'].dt.year
        
        # Drop all columns that are not in 'columns_to_keep'
        filtered_df = df[columns_to_keep]
        missing_weeks_data.append(filtered_df)
    
    return missing_weeks_data

In [4]:
# fetch team totals and O/U data for multiple weeks (imports json files)
def teamTotals_overUnder():
    all_weeks_data = []

    # fetch data for each week (you can adjust the range as needed)
    for week in range(1, 18):  # adjust range based on the weeks you want to fetch
        df = import_rotowire_data(week)
        if df is not None:
            all_weeks_data.append(df)

    # load missing weeks from local JSON files and append them
    missing_weeks_data = load_json_files()
    all_weeks_data.extend(missing_weeks_data)

    # concatenate all non-empty DataFrames
    if all_weeks_data:
        full_season_data = pd.concat(all_weeks_data, ignore_index=True)

        # sort by 'year' (descending) and 'week' (ascending)
        full_season_data = full_season_data.sort_values(by=['year', 'week'], ascending=[False, True])

        # display the sorted DataFrame
        # display(full_season_data)

        # save the sorted DataFrame to a CSV file
        # csv_filename = "teamTotals_overUnder.csv"
        # full_season_data.to_csv(csv_filename, index=False)
        # print(f"Data successfully written to {csv_filename}")
    else:
        print("No data fetched for any week.")

# Call the function to execute the workflow
# teamTotals_overUnder()

In [5]:
# fetch team totals and O/U data for the current week only
# *** IMPORTANT***: update the NFL season start date for each new season 
def get_current_week():
    
    # Logic to compute the current NFL week
    current_date = datetime.now()
    
    # Define the start date of Week 1 in the NFL season (adjust this as needed)
    season_start_date = datetime(2024, 9, 4)
    
    # Calculate the week number based on the difference from the start date
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# **** IMPORTANT: the qb_and_team_betting_lines() functions needs the csv file to create the df
def teamTotals_overUnder_current_week():
    current_week_data = []

    # Define the current week (you can adjust this to get the actual current week dynamically)
    current_week = get_current_week() 

    # Fetch data for the current week
    df = import_rotowire_data(current_week)
    if df is not None:
        current_week_data.append(df)

    # Concatenate all non-empty DataFrames
    if current_week_data:
        full_current_week_data = pd.concat(current_week_data, ignore_index=True)

        # Sort by 'year' (descending) and 'week' (ascending)
        full_current_week_data = full_current_week_data.sort_values(by=['year', 'week'], ascending=[False, True])

        # Display the sorted DataFrame
        # display(full_current_week_data)

        # Save the sorted DataFrame to a CSV file   - DO NOT COMMENT OUT
        csv_filename = "teamTotals_overUnder_current_week.csv"
        full_current_week_data.to_csv(csv_filename, index=False)
        print(f"Data successfully written to {csv_filename}")
    else:
        print("No data fetched for the current week.")

# Call the new function to fetch data for the current week
# teamTotals_overUnder_current_week()

In [6]:
# test
# Function 1: Fetch QB Player Data from ESPN API
def import_ESPN_qb_data(year, seasontype):
    
    # Define the ESPN API URL with parameters
    espn_url = f"https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page=1&limit=50&category=offense%3Apassing&sort=passing.passingYards%3Adesc&season={year}&seasontype={seasontype}"

    # Send the request
    response = requests.get(espn_url)

    # Check the response status and parse JSON data
    if response.status_code == 200:
        data = response.json()

        # Extract player information, including shortName, team name, and team abbreviation
        player_list = []
        for athlete_data in data['athletes']:
            athlete = athlete_data['athlete']
            
            # Extract necessary fields
            player_name = athlete.get('displayName', 'N/A')
            short_name = athlete.get('shortName', 'N/A')
            player_id = athlete.get('id', 'N/A')
            team_name = athlete.get('teamName', 'N/A')
            team_abbreviation = athlete.get('teamShortName', 'N/A')
            
            # Adding team details
            player_info = {
                'name': player_name,
                'shortName': short_name,
                'id': player_id,
                'teams': [
                    {
                        'name': team_name,
                        'abbreviation': team_abbreviation
                    }
                ]
            }
            player_list.append(player_info)

        return player_list
    else:
        print(f"Error: Failed to fetch data, status code {response.status_code}")
        return []

In [7]:
# test
# Function 2: Merge QB Data with Betting Lines Data
def create_qb_betting_lines_df(player_list, csv_file_path):
 
    # Convert player_list to a DataFrame
    player_df = pd.DataFrame([{
        'name': player['name'],
        'shortName': player['shortName'],
        'id': player['id'],
        'team_name': player['teams'][0]['name'],
        'team_abbreviation': player['teams'][0]['abbreviation']
    } for player in player_list])

    # Load the CSV data
    betting_df = pd.read_csv(csv_file_path)

    # Display the columns of the betting data
    # print("Betting Data Columns:\n", betting_df.columns)

    # Proceed with the merge if the data is valid
    if not betting_df.empty:
        merged_df = pd.merge(player_df, betting_df, how='left', 
                             left_on=['team_name', 'team_abbreviation'], 
                             right_on=['nickname', 'abbr'])

        # Return the merged data
        return merged_df
    else:
        print("Error: Betting data is empty.")
        return None

In [8]:
# **IMPORTANT: this function outputs the combined qb-betting lines df
# **Use this function to make further modifications
# Function 3: Main function to execute the full workflow
def qb_and_team_betting_lines():
    # Get the current NFL week
    current_week = get_current_week()
    
    # Logic to determine the year and season type based on the current week
    current_year = datetime.now().year
    if current_week <= 18:
        year = current_year
        seasontype = 2  # Regular season
    elif 18 <= current_week <= 21:
        year = current_year
        seasontype = 3  # Playoffs
    else:
        year = current_year - 1  # If week is after playoffs, default to last season
        seasontype = 2  # Regular season

    print(f"Year: {year}, Season Type: {seasontype}, Current Week: {current_week}")
    
    # Check to see if the .csv file is in the directory
    csv_file_path = 'teamTotals_overUnder_current_week.csv'
    if os.path.exists(csv_file_path):
        print(f"{csv_file_path} found.")
    else:
        print(f"{csv_file_path} not found, fetching betting lines data...")
        teamTotals_overUnder_current_week()  # Fetch betting lines if CSV not found

    # Fetch the player data
    qb_player_data = import_ESPN_qb_data(year, seasontype)

    # Merge the player data with betting lines data
    if qb_player_data:
        qb_betting_lines_merged_data = create_qb_betting_lines_df(qb_player_data, csv_file_path)

        # Display the merged data
        if qb_betting_lines_merged_data is not None:
            print("Merged Data:\n")
            
            # Display the merged DataFrame
            # display(qb_betting_lines_merged_data)  

            # Save the merged DataFrame to a CSV file with the same name as the function
            # output_csv_file = 'qb_and_team_betting_lines.csv'
            # qb_betting_lines_merged_data.to_csv(output_csv_file, index=False)
            # print(f"Data successfully written to {output_csv_file}")

        # Return the merged DataFrame for further use
        return qb_betting_lines_merged_data  
    else:
        print("Error: No player data found.")
        return None

# Call the main function to execute the workflow
qb_betting_lines_df = qb_and_team_betting_lines()

# Optional: Use qb_betting_lines_df for further processing if returned correctly
# if qb_betting_lines_df is not None:
#     print("Successfully merged QB data with betting lines and saved to CSV.")


Year: 2024, Season Type: 2, Current Week: 5
teamTotals_overUnder_current_week.csv found.
Merged Data:



In [9]:
# returns the qb and corresponding betting lines
# Modifies the qb_betting_lines_df DataFrame by dropping, renaming, and reordering columns
def modify_qb_betting_lines_df(qb_betting_lines_df):

    # Step 1: Drop the specified columns
    columns_to_drop = ['gameDateTime', 'shortName', 'team_abbreviation', 'nickname', 'id']
    qb_betting_lines_df = qb_betting_lines_df.drop(columns=columns_to_drop, errors='ignore')

    # Step 2: Rename columns
    qb_betting_lines_df = qb_betting_lines_df.rename(columns={
        'oppNickname': 'opponent',
        'team_name': 'team'
    })

    # Step 3: Check which columns exist for reordering
    column_order = ['year', 'week', 'gameID', 'gameDate', 'name', 'id', 'team', 'abbr', 'homeAway']
    existing_columns = [col for col in column_order if col in qb_betting_lines_df.columns]
    remaining_columns = [col for col in qb_betting_lines_df.columns if col not in existing_columns]

    # Apply the order with the existing columns
    qb_betting_lines_df = qb_betting_lines_df[existing_columns + remaining_columns]

    # Step 4: Display the modified DataFrame
    print("Modified DataFrame:\n")
    display(qb_betting_lines_df)

    # Step 5: Save the modified DataFrame to a CSV file
    output_csv_file = 'qb_betting_lines.csv'
    qb_betting_lines_df.to_csv(output_csv_file, index=False)
    print(f"Modified DataFrame successfully written to {output_csv_file}")

    return qb_betting_lines_df

# Call the function to modify the qb_betting_lines_df
modified_qb_betting_lines_df = modify_qb_betting_lines_df(qb_betting_lines_df)


Modified DataFrame:



,year,week,gameID,gameDate,name,team,abbr,homeAway,opponent,draftkings_moneyline,...,betrivers_moneyline,betrivers_spread,betrivers_ou,betrivers_score,betrivers_oppScore,mgm_moneyline,mgm_spread,mgm_ou,mgm_score,mgm_oppScore
0,NaN,NaN,NaN,NaN,Kirk Cousins,Falcons,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024.0,5.0,2656687.0,2024-10-06 16:25:00,Geno Smith,Seahawks,SEA,home,Giants,-325.0,...,-315.0,-7.0,42.5,24.8,17.8,-350.0,-7.0,43.0,25.0,18.0
2,NaN,NaN,NaN,NaN,Baker Mayfield,Buccaneers,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024.0,5.0,2656673.0,2024-10-06 16:05:00,Brock Purdy,49ers,SF,home,Cardinals,-355.0,...,-375.0,-7.0,50.0,28.5,21.5,-350.0,-7.5,50.0,28.8,21.3
4,2024.0,5.0,2656690.0,2024-10-06 20:20:00,Dak Prescott,Cowboys,DAL,away,Steelers,124.0,...,125.0,2.5,44.0,20.8,23.3,120.0,2.5,44.0,20.8,23.3
5,2024.0,5.0,2656655.0,2024-10-06 13:00:00,C.J. Stroud,Texans,HOU,home,Bills,-115.0,...,-117.0,-1.0,47.0,24.0,23.0,-110.0,-1.0,47.5,24.3,23.3
6,NaN,NaN,NaN,NaN,Jared Goff,Lions,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2024.0,5.0,2656683.0,2024-10-06 16:25:00,Matthew Stafford,Rams,LAR,home,Packers,136.0,...,135.0,3.0,48.5,22.8,25.8,145.0,3.0,48.5,22.8,25.8
8,2024.0,5.0,2656653.0,2024-10-06 13:00:00,Joe Burrow,Bengals,CIN,home,Ravens,114.0,...,120.0,2.5,49.0,23.3,25.8,115.0,2.5,49.5,23.5,26.0
9,2024.0,5.0,2656571.0,2024-10-06 09:30:00,Sam Darnold,Vikings,MIN,home,Jets,-142.0,...,-141.0,-2.5,40.5,21.5,19.0,-145.0,-2.5,40.5,21.5,19.0


Modified DataFrame successfully written to qb_betting_lines.csv


In [10]:
def qb_teamTotal_23pts_OU_45(modified_qb_betting_lines_df):
    """
    Filters the modified DataFrame to return rows where:
    - The average projected O/U is at least 45
    - The projected team total is at least 23 points.
    
    It also adds new columns for average O/U and average team total, and returns 
    the DataFrame with the specified columns and order.
    """

    # Step 1: Compute the average O/U for each team
    ou_columns = ['draftkings_ou', 'fanduel_ou', 'betrivers_ou', 'mgm_ou']
    modified_qb_betting_lines_df['avg_over_under'] = modified_qb_betting_lines_df[ou_columns].mean(axis=1)

    # Step 2: Compute the average team total for each team
    score_columns = ['draftkings_score', 'fanduel_score', 'betrivers_score', 'mgm_score']
    modified_qb_betting_lines_df['avg_team_total'] = modified_qb_betting_lines_df[score_columns].mean(axis=1)

    # Step 3: Filter rows where avg O/U is at least 45 and avg team total is at least 23
    filtered_df = modified_qb_betting_lines_df[
        (modified_qb_betting_lines_df['avg_over_under'] >= 45) &
        (modified_qb_betting_lines_df['avg_team_total'] >= 23)
    ]

    # Step 4: Reorder columns
    column_order = ['year', 'week', 'gameID', 'gameDate', 'name', 'team', 'abbr', 'homeAway', 'opponent', 'avg_over_under', 'avg_team_total']
    filtered_df = filtered_df[column_order]

    # Display the final filtered DataFrame
    print("Filtered DataFrame with avg O/U >= 45 and avg team total >= 23:\n")
    display(filtered_df)

    # Save the filtered DataFrame to a CSV file
    output_csv_file = 'qb_teamTotal_23pts_OU_45.csv'
    filtered_df.to_csv(output_csv_file, index=False)
    print(f"Filtered DataFrame successfully written to {output_csv_file}")

    return filtered_df
    
# Call the new function on the modified DataFrame
qb_team_total_df = qb_teamTotal_23pts_OU_45(modified_qb_betting_lines_df)

Filtered DataFrame with avg O/U >= 45 and avg team total >= 23:



,year,week,gameID,gameDate,name,team,abbr,homeAway,opponent,avg_over_under,avg_team_total
3,2024.0,5.0,2656673.0,2024-10-06 16:05:00,Brock Purdy,49ers,SF,home,Cardinals,49.875,28.650
5,2024.0,5.0,2656655.0,2024-10-06 13:00:00,C.J. Stroud,Texans,HOU,home,Bills,47.250,24.200
8,2024.0,5.0,2656653.0,2024-10-06 13:00:00,Joe Burrow,Bengals,CIN,home,Ravens,49.000,23.275
15,2024.0,5.0,2656653.0,2024-10-06 13:00:00,Lamar Jackson,Ravens,BAL,away,Bengals,49.000,25.775
19,2024.0,5.0,2656655.0,2024-10-06 13:00:00,Josh Allen,Bills,BUF,away,Texans,47.250,23.075
22,2024.0,5.0,2656659.0,2024-10-06 13:00:00,Trevor Lawrence,Jaguars,JAX,home,Colts,46.000,24.525
26,2024.0,5.0,2656683.0,2024-10-06 16:25:00,Jordan Love,Packers,GB,away,Rams,48.500,25.800
32,2024.0,5.0,2656683.0,2024-10-06 16:25:00,Malik Willis,Packers,GB,away,Rams,48.500,25.800
40,2024.0,5.0,2656655.0,2024-10-06 13:00:00,Davis Mills,Texans,HOU,home,Bills,47.250,24.200
43,2024.0,5.0,2656659.0,2024-10-06 13:00:00,Mac Jones,Jaguars,JAX,home,Colts,46.000,24.525


Filtered DataFrame successfully written to qb_teamTotal_23pts_OU_45.csv


In [11]:
def qb_teamTotal_23pts_OU_45_7pt_favorite(modified_qb_betting_lines_df):
    """
    Filters the modified DataFrame to return rows where:
    - The average projected O/U is at least 45
    - The projected team total is at least 23 points
    - The team is favored by at least 7 points (spread <= -7).
    
    It also adds new columns for average O/U, average team total, and average spread,
    and returns the DataFrame with the specified columns and order.
    """

    # Step 1: Compute the average O/U for each team
    ou_columns = ['draftkings_ou', 'fanduel_ou', 'betrivers_ou', 'mgm_ou']
    modified_qb_betting_lines_df['avg_over_under'] = modified_qb_betting_lines_df[ou_columns].mean(axis=1)

    # Step 2: Compute the average team total for each team
    score_columns = ['draftkings_score', 'fanduel_score', 'betrivers_score', 'mgm_score']
    modified_qb_betting_lines_df['avg_team_total'] = modified_qb_betting_lines_df[score_columns].mean(axis=1)

    # Step 3: Compute the average projected spread for each team
    spread_columns = ['draftkings_spread', 'fanduel_spread', 'betrivers_spread', 'mgm_spread']
    modified_qb_betting_lines_df['average_spread'] = modified_qb_betting_lines_df[spread_columns].mean(axis=1)

    # Step 4: Filter rows based on conditions
    filtered_df = modified_qb_betting_lines_df[
        (modified_qb_betting_lines_df['avg_over_under'] >= 45) &  # avg O/U at least 45
        (modified_qb_betting_lines_df['avg_team_total'] >= 23) &  # avg team total at least 23
        (modified_qb_betting_lines_df['average_spread'] <= -7)    # avg spread <= -7 (favored by at least 7 points)
    ]

    # Step 5: Reorder columns
    column_order = ['year', 'week', 'gameID', 'gameDate', 'name', 'team', 'abbr', 'homeAway', 'opponent', 'avg_over_under', 'avg_team_total', 'average_spread']
    filtered_df = filtered_df[column_order]

    # Display the final filtered DataFrame
    print("Filtered DataFrame with avg O/U >= 45, avg team total >= 23, and avg spread <= -7:\n")
    display(filtered_df)

    # Save the filtered DataFrame to a CSV file
    output_csv_file = 'qb_teamTotal_23pts_OU_45_7pt_fav.csv'
    filtered_df.to_csv(output_csv_file, index=False)
    print(f"Filtered DataFrame successfully written to {output_csv_file}")

    return filtered_df
    
# Call the new function on the modified DataFrame
qb_team_total_7pt_fav_df = qb_teamTotal_23pts_OU_45_7pt_favorite(modified_qb_betting_lines_df)

Filtered DataFrame with avg O/U >= 45, avg team total >= 23, and avg spread <= -7:



,year,week,gameID,gameDate,name,team,abbr,homeAway,opponent,avg_over_under,avg_team_total,average_spread
3,2024.0,5.0,2656673.0,2024-10-06 16:05:00,Brock Purdy,49ers,SF,home,Cardinals,49.875,28.65,-7.375


Filtered DataFrame successfully written to qb_teamTotal_23pts_OU_45_7pt_fav.csv
